# Agent

In [ ]:
import getpass
import os
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)

In [ ]:
search.invoke("que es openai o1-mini")

In [ ]:
!pip -qU install faiss-gpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("how to upload a dataset")[0]

In [ ]:
from langchain.tools.retriever import create_retriever_tool

txt = '''Busca información sobre LangSmith.
¡Para cualquier pregunta sobre LangSmith,
debes usar esta herramienta!'''

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    txt,)

In [ ]:
tools = [search, retriever_tool]

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content=
                            "What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
agent_executor.invoke({"input": "hi!"})

In [ ]:
agent_executor.invoke({"input": "Cómo ayuda langsmith testing?"})

In [ ]:
print(agent_executor.invoke({"input": "Como intalar LangSmith?"})['output'])

In [ ]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "Mi nombre es Bob", "chat_history": []})

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="Mi nombre es Bob"),
            AIMessage(content="Hola Bob! ¿En qué puedo ayudarte hoy?"),
        ],
        "input": "Cuál es mi nombre?",
    }
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)